# Pill Working Example
https://arxiv.org/abs/2012.02904

The justification is that `(onDay pill Wednesday) (beforeTime pill afternoon)`. And the final explanation reads in a series of symbolic triples: `(prefers user (before pill activity)) (IsA user activity) (atTime appt 1pm) (onDay appt Wednesday) (IsA 1pm afternoon)`.

In [1]:
import commonsense.conceptnet as kb
from commonsense.logical_classes import Fact, to_data_frame, parse_file_to_fact_list, parse_file_to_event_list, parse_raw_file_to_event_list, create_facts_from_file
from monitor.reasonableness_monitor import SnapshotMonitor

In [2]:
events = [Fact('Levodopa', 'onDate', 'Wednesday')]
preference = Fact('pill', 'before', 'activity')

cn = kb.ConceptNet()
pill_monitor = SnapshotMonitor()

additional_facts = create_facts_from_file("pill_facts.txt")  # additional information
pill_monitor.explain(events, additional_facts, preference)

                      subject predicate        object      reason  score  \
0                   wednesday       IsA     a weekday  ConceptNet    1.0   
1                   wednesday       IsA       weekday  ConceptNet    1.0   
2  the fourth day of the week       IsA     wednesday  ConceptNet    1.0   
3                   wednesday       IsA       weekday  ConceptNet    1.0   
4                   wednesday       IsA  business day  ConceptNet    1.0   
5                   wednesday       IsA           day  ConceptNet    1.0   

   count  
0      1  
1      1  
2      1  
3      1  
4      1  
5      1  
                       subject predicate        object      reason  score  \
0                    wednesday       IsA     a weekday  ConceptNet    1.0   
1                    wednesday       IsA       weekday  ConceptNet    1.0   
2   the fourth day of the week       IsA     wednesday  ConceptNet    1.0   
3                    wednesday       IsA       weekday  ConceptNet    1.0   
4    

AttributeError: 'SnapshotMonitor' object has no attribute 'chain'

In [18]:
from reasoner.production import IF, AND, OR, NOT, THEN, DELETE, forward_chain, pretty_goal_tree, match, populate, simplify
from reasoner.data import *
import pprint

transitive_rule = IF(AND('(?x) IsA (?y)', '(?y) IsA (?z)'),
                     THEN('(?x) IsA (?z)'))

# TODO: Get penguin facts
# TODO: Get rules


# Small examples for Demo


In [19]:
import pprint 

pp = pprint.PrettyPrinter(indent=1)
pprint = pp.pprint

query = Fact('penguin', 'IsA', 'animal')

cn = kb.ConceptNet()
monitor = SnapshotMonitor()

facts = monitor.explain_fact(query)
data_facts = [fact.to_string() for fact in facts]
print("started with %s facts"%len(data_facts))
#print(facts)

chained_data = forward_chain([transitive_rule], data_facts)

print("ended with %s facts"%len(chained_data))

started with 211 facts
ended with 213 facts


In [23]:
def backchain_to_goal_tree(rules, hypothesis):
    """
    Takes a hypothesis (string) and a list of rules (list
    of IF objects), returning an AND/OR tree representing the
    backchain of possible statements we may need to test
    to determine if this hypothesis is reachable or not.

    This method should return an AND/OR tree, that is, an
    AND or OR object, whose constituents are the subgoals that
    need to be tested. The leaves of this tree should be strings
    (possibly with unbound variables), *not* AND or OR objects.
    Make sure to use simplify(...) to flatten trees where appropriate.
    """
    top_level = [hypothesis]
    
    for rule in rules:
        matching = match(rule.consequent(), hypothesis)
    
        if matching is not None:
            tree = []
            next = populate(rule.antecedent(), matching)
            ant_stuff = type(rule.antecedent())
    
            if ant_stuff is str:  # leaf
                tree.append(backchain_to_goal_tree(rules, next))
            else:  # it's a phrase
                for hyp in next:
                    tree.append(backchain_to_goal_tree(rules, hyp))
    
            if ant_stuff == OR:
                top_level.append(OR(tree))
            else:
                top_level.append(AND(tree))
    return simplify(OR(top_level))

# pretty_goal_tree(backchain_to_goal_tree(zookeeper_rules, 'opus is a penguin'))
# result = backchain_to_goal_tree(zookeeper_rules, 'opus is a penguin')

pretty_goal_tree(backchain_to_goal_tree([transitive_rule], 'penguin IsA animal'))

KeyError: 'y'

In [ ]:
import pprint 

pp = pprint.PrettyPrinter(indent=1)
pprint = pp.pprint

query = Fact('penguin', 'IsA', 'animal')

cn = kb.ConceptNet()
monitor = SnapshotMonitor()

facts = monitor.explain_fact(query)
# data_facts = [fact.to_string() for fact in facts]
# print("started with %s facts"%len(data_facts))
#print(facts)

chained_data = forward_chain([transitive_rule], facts)

print("ended with %s facts"%len(chained_data))

# Lego example


In [ ]:
from commonsense.logical_classes import preprocess

events = preprocess()  # This reads in the file: allnewsemanticdatawithmultimodal0025.txt by default
cn = kb.ConceptNet()
cn_monitor = SnapshotMonitor()

cn_monitor.explain(events, additional_facts)

# Older Lego Example

In [ ]:
import commonsense.conceptnet as kb
from commonsense.logical_classes import to_data_frame, parse_file_to_fact_list, parse_file_to_event_list, parse_raw_file_to_event_list, create_facts_from_file
from monitor.reasonableness_monitor import SnapshotMonitor

cn = kb.ConceptNet()  # choose your KB 
cn_monitor = SnapshotMonitor()  # labels=labels, data=data, rules=rules)
print("Made a ConceptNet monitor, now trying to explain")

events = parse_file_to_event_list("datasets/PAX/output_feb25.txt")  # This reads in the file. 
additional_facts = create_facts_from_file("gaze_facts.txt") # should call rules 
cn_monitor.explain_events(events, additional_facts)


In [ ]:
facts

f = to_data_frame(facts)
events = f.groupby("subject")
f.groupby("subject").count()
events.first()
events.nth(0)

facts